[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nils-holmberg/socs-qmd/blob/main/jnb/lab6_cv3.ipynb)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# image dataset

In [ ]:
!gdown https://drive.google.com/uc?id=1sBsckDIyQJ-zTsEpjCntYMIJcbBpF3wk

In [ ]:
!unzip content-images.zip

# video dataset

In [ ]:
!gdown https://drive.google.com/uc?id=1T9S32UwmDnUd6YbxRYfyG7-gud8jh2Y_


In [ ]:
import cv2
import os

# Path to the video file
video_path = 'content-video.mp4'

# Create a VideoCapture object
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Directory to save frames
save_dir = 'images-video'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

frame_idx = 0
while True:
    ret, frame = cap.read()

    # Break the loop if there are no more frames
    if not ret:
        break

    # Save every 100th frame
    if frame_idx % 100 == 0:
        # Filename with 5 leading zeroes
        save_path = os.path.join(save_dir, f'frame_{frame_idx:05d}.jpg')
        cv2.imwrite(save_path, frame)
        print(f'Saved frame {frame_idx:05d}')

    frame_idx += 1

# Release the VideoCapture object
cap.release()


# image normalization

In [ ]:
import cv2
import os
import numpy as np

def resize_and_crop(img, size):
    # Resize image to maintain aspect ratio
    h, w, _ = img.shape
    if h > w:
        new_h, new_w = size * h / w, size
    else:
        new_h, new_w = size, size * w / h

    new_h, new_w = int(new_h), int(new_w)
    resized_img = cv2.resize(img, (new_w, new_h))

    # Crop the center of the image
    startx = new_w//2 - size//2
    starty = new_h//2 - size//2
    return resized_img[starty:starty+size, startx:startx+size]

def normalize_image(img):
    # Normalize pixel values to [0, 1]
    return img / 255.0

directory = 'images-video'
output_directory = 'images-normalize'

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for filename in os.listdir(directory):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
#    if filename.endswith('.jpg'):
        img = cv2.imread(os.path.join(directory, filename))
        img = resize_and_crop(img, 256)
        normalized_img = normalize_image(img)

        # Convert the normalized image back to 8-bit format
        img_to_save = (normalized_img * 255).astype(np.uint8)

        # Save the normalized image
        output_path = os.path.join(output_directory, filename)
        cv2.imwrite(output_path, img_to_save)


# object features

- face object detection with haar features

In [ ]:
import cv2

# Load the cascade
#face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
face_cascade=cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
# Read the input image
img = cv2.imread('/content/images-video/frame_01600.jpg')
# Convert into grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Detect faces
faces = face_cascade.detectMultiScale(gray, 1.1, 4)
# Draw rectangle around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
# Display the output
#cv2.imshow('img', img)
#cv2.waitKey()
from google.colab.patches import cv2_imshow
cv2_imshow(img)

In [ ]:
cv2.imwrite("haar-face.png", img)

# image classification

- deep learning, mnist dataset

## intro

![mnist](https://raw.githubusercontent.com/nils-holmberg/sfac-py/main/fig/mnist-digits.png)

import packages

In [ ]:
#!pip install numpy
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

%matplotlib inline


In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
#import tensorflow as tf; print(tf.__version__)

## data collection, preprocessing

In [ ]:
# load the MNIST dataset
(x_train, y_trainO), (x_test, y_testO) = mnist.load_data()
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
num_classes = 10
y_train = keras.utils.to_categorical(y_trainO, num_classes)
y_test = keras.utils.to_categorical(y_testO, num_classes)

# visualize data
plt.rcParams['figure.figsize'] = (6,6)
obs_idx = 1000
plt.imshow(x_train[obs_idx], cmap='gray')
print(y_train[obs_idx])

# convert images into vectors of numbers
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# normalize
x_train /= 255
x_test /= 255


## model specification

In [ ]:
# specify MLP model (multi-layer perceptron)
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(784,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

# compile model
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])


# deep learning model

![mnist](https://raw.githubusercontent.com/nils-holmberg/sfac-py/main/fig/lecture-201116-aiml.png)


## train model

- multilayer perceptron (mlp)

In [ ]:
batch_size = 128
epochs = 10

# train the model (output training history)
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

# training diagnostics
plt.figure('Model training')
plt.ylabel('training error')
plt.xlabel('epoch')
for k in history.history.keys():
    plt.plot(history.history[k], label=k)
plt.legend(loc='best')

## evaluate model

- on test data, generalization

In [ ]:
# evaluate model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# make a prediction on the test set
#predicted_classes = model.predict_classes(x_test)
predicted_classes = np.argmax(model.predict(x_test), axis=-1)

# see which we predicted correctly and which not
correct_indices = np.nonzero(predicted_classes == y_testO)[0]
incorrect_indices = np.nonzero(predicted_classes != y_testO)[0]
print()
print(len(correct_indices), "classified correctly")
print(len(incorrect_indices), "classified incorrectly")


In [ ]:
def plot_confusion_matrix(y_true, y_pred, cmap=plt.cm.Blues):
    """
    The function is used to construct the confusion matrix
    """
    cm = confusion_matrix(y_true, y_pred)
    fig = plt.figure(figsize=(10,10))
    plt.rcParams.update({'font.size': 16})
    ax  = fig.add_subplot(111)
    matrix = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    fig.colorbar(matrix)
    for i in range(0,10):
        for j in range(0,10):
            ax.text(j,i,cm[i,j],va='center', ha='center')
    ticks = np.arange(len(labels))
    ax.set_xticks(ticks)
    ax.set_xticklabels(labels, rotation=45)
    ax.set_yticks(ticks)
    ax.set_yticklabels(labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
# visualize true and predicted labels
y_prob = model.predict(x_test, batch_size=32, verbose=0)
y_pred = [np.argmax(prob) for prob in y_prob]
y_true = [np.argmax(true) for true in y_test]
labels = np.arange(0,10,1)
plot_confusion_matrix(y_true, y_pred, cmap=plt.cm.YlOrRd_r)


In [ ]:
# Some visual feedback

# Make a prediction on the test set
predicted_classes = np.argmax(model.predict(x_test), axis=-1)

# See which we predicted correctly and which not
correct_indices = np.nonzero(predicted_classes == y_testO)[0]
incorrect_indices = np.nonzero(predicted_classes != y_testO)[0]
print()
print(len(correct_indices)," classified correctly")
print(len(incorrect_indices)," classified incorrectly")

# Adapt figure size to accomodate 18 subplots
plt.rcParams['figure.figsize'] = (20,40)

figure_evaluation = plt.figure()

# Plot 9 correct predictions
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(6,3,i+1)
    plt.imshow(x_test[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title(
      "Predicted: {}, Truth: {}".format(predicted_classes[correct],
                                        y_testO[correct]))
    plt.xticks([])
    plt.yticks([])

# Plot 9 incorrect predictions
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(6,3,i+10)
    plt.imshow(x_test[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title(
      "Predicted {}, Truth: {}".format(predicted_classes[incorrect],
                                       y_testO[incorrect]))
    plt.xticks([])
    plt.yticks([])


In [ ]:
#model.save("mnist.mod")
model.save('mnist.h5')

# image object detection

- [huggingface](https://huggingface.co/tasks)

In [ ]:
!pip install -q transformers
!pip install -q timm


In [ ]:
from transformers import pipeline
model = pipeline("object-detection")

In [ ]:
import glob

rows_list = []
for img in sorted(glob.glob('images-video/frame_*.jpg')):
  dict1 = {}
  res = model(img)
  dict1.update({"image": img, "result": res})
  rows_list.append(dict1)

df = pd.DataFrame(rows_list)
df.head()

In [ ]:
rows_list = []
for index, row in df.iterrows():
  for i in row.result:
    dict1 = {}
    dict1.update({"image":row.image})
    dict1.update(i)
    rows_list.append(dict1)

df_res = pd.DataFrame(rows_list)
df_res.head()

In [ ]:
df_res.label.value_counts().plot(kind='bar', figsize=(6, 4))

In [ ]:
df_res[df_res.image=="images-video/frame_00100.jpg"]

In [ ]:
import cv2

img = cv2.imread('images-video/frame_00100.jpg')
for index,row in df_res[df_res.image=="images-video/frame_00100.jpg"].iterrows():
  x1,y1,x2,y2 = list(row.box.values())
  cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

from google.colab.patches import cv2_imshow
cv2_imshow(img)
#cv2.imshow("display", img)
#cv2.imwrite("objects.png", img)

# advanced content inference

- [mediapipe](https://www.youtube.com/watch?v=01sAkU_NvOY)

In [ ]:
!pip install -q mediapipe

In [ ]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=1, enable_segmentation=True)
mp_drawing = mp.solutions.drawing_utils

# Read an image
image_path = 'images-video/frame_02800.jpg'  # Replace with the path to your image
image_path = 'images-video/frame_00400.jpg'  # Replace with the path to your image
image = cv2.imread(image_path)

# Check if image is loaded
if image is None:
    print("Error: Image not found.")
else:
    # Convert the BGR image to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image and find the pose
    results = pose.process(image_rgb)

    # Draw pose landmarks on the image
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Convert back to BGR for displaying with OpenCV
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Display the image
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# Release resources
pose.close()


# try detectron2

- [facebook meta](https://ai.meta.com/tools/detectron2/)
- requires gpu session

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
img = cv2.imread('/content/images-video/frame_00500.jpg')
cv2_imshow(img)

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(img)

In [ ]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

In [ ]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
# Inference with a keypoint detection model
cfg = get_cfg()   # get a fresh new config
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(img)
v = Visualizer(img[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
# Inference with a panoptic segmentation model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
predictor = DefaultPredictor(cfg)
panoptic_seg, segments_info = predictor(img)["panoptic_seg"]
v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
cv2_imshow(out.get_image()[:, :, ::-1])

# video analysis

In [ ]:
# load data from url
#import urllib.request
#url = "https://raw.githubusercontent.com/nils-holmberg/cca-cv/main/vid/clip.mp4"
#fp = "/content/content-video.mp4"
#urllib.request.urlretrieve(url, fp)
!ffmpeg -i /content/content-video.mp4 -t 00:00:06 -c:v copy /content/content-video-clip.mp4

In [ ]:
# Run frame-by-frame inference demo on this video (takes 3-4 minutes) with the "demo.py" tool we provided in the repo.
!git clone https://github.com/facebookresearch/detectron2
# Note: this is currently BROKEN due to missing codec. See https://github.com/facebookresearch/detectron2/issues/2901 for workaround.
%run detectron2/demo/demo.py --config-file detectron2/configs/COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml --video-input /content/content-video-clip.mp4 --confidence-threshold 0.6 --output content-video-clip.mkv \
  --opts MODEL.WEIGHTS detectron2://COCO-PanopticSegmentation/panoptic_fpn_R_101_3x/139514519/model_final_cafdb1.pkl

In [ ]:
# Download the results
from google.colab import files
files.download('content-video-clip.mkv')